In [2]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span,ColorBar, LogColorMapper
from bokeh.palettes import viridis



# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
def run_model(model,t,parameters_list,molecules_0,dilute_list,replenish_list,result_all,DR=0.2):  
    start_cycle,end_cycle = np.array(t)*4 #turn hour to cycle
    for n in range (start_cycle,end_cycle):
        #define time
        t_start= n*15
        t_end = (n+1)*15
        t= np.linspace(t_start,t_end,2)

        #solve equation and save result
        result = scipy.integrate.odeint(model, molecules_0, t, args=parameters_list)
        result_all = np.append(result_all,result[1])
        
        #update parameter
        molecules_0 = result.transpose()[:,-1]
        
        #dilution 
        ###diute out
        dilute_species((dilute_list),molecules_0,DR)
        
        ###replenish 
        replenish_species((replenish_list),molecules_0,DR)
    return result_all,molecules_0
    


In [4]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.1
T7_DNA.lc =0.1

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0
Repressor_DNA.lc = 0


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [12]:
#experiment 
T7_DNA.ic = 0.1
T7_DNA.lc = 0.1
#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


#######create plot#############
p = bokeh.plotting.figure(
    plot_width=1000,
    plot_height=500,
    x_axis_label="t",
    y_axis_type="linear",
    title="T7_DNA"
)


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition



DR = 0.2
color_num = 0

condition_low_log = -4
condition_high_log = 2

GFP_15hr = []

for T7_DNA_conc in np.logspace(condition_low_log,condition_high_log,50):
    color_num += 1
    parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)

    T7_DNA.ic = T7_DNA_conc
    T7_DNA.lc = T7_DNA_conc

    #initial value
    molecules_0 = [None]*len(molecules_list) #create empty list

    for i in range(0,len(molecules_list)):
        molecules_0[i] = molecules_list[i].ic
    #stage 1 kick start
    result_all=[molecules_0] #initial condition

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all,DR)


    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all,DR)



    result_all = np.resize(result_all,(121,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]

    #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()

    t = np.linspace(0, 15*(len(GFP.result)+1), len(GFP.result))


    colors = bokeh.palettes.d3["Category10"][3]

    # Populate glyphs
    p.line(
        t/60, GFP.result, line_width=4, color=viridis(51)[color_num],alpha=0.8
    )
    vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
    p.add_layout(vline1)
    GFP_15hr = np.append(GFP_15hr,GFP.result[60])


color_mapper = LogColorMapper(palette="Viridis256", low=10**condition_low_log, high=10**condition_high_log)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)
p.add_layout(color_bar, 'right')



show(p)

In [85]:
conc = np.logspace(-4,1, len(GFP_15hr), endpoint=True)

#######create plot#############
p = bokeh.plotting.figure(
    plot_width=1000,
    plot_height=500,
    x_axis_label="T7_DNA_conc",
    x_axis_type = "log",
    y_axis_label="GFP",
)
p.scatter(
        conc, GFP_15hr, line_width=4
    )

show(p)

In [80]:
GFP_15hr

array([1.42172491e+00, 1.48983348e+00, 1.58399951e+00, 1.71589633e+00,
       1.90376471e+00, 2.17714962e+00, 2.58589294e+00, 3.21796725e+00,
       4.23648850e+00, 5.96002645e+00, 9.04340269e+00, 1.48884078e+01,
       2.64886381e+01, 4.90033245e+01, 7.65241354e+01, 7.88527394e+01,
       7.64687746e+01, 7.37695438e+01, 7.08601910e+01, 6.77474391e+01,
       6.44406617e+01, 6.09535073e+01, 5.73037707e+01, 5.35130785e+01,
       4.96064750e+01, 4.56120258e+01, 4.15605922e+01, 3.74859285e+01,
       3.34251944e+01, 2.94198295e+01, 2.55164858e+01, 2.17674077e+01,
       1.82293947e+01, 1.49605330e+01, 1.20144550e+01, 9.43298474e+00,
       7.23919774e+00, 5.43334929e+00, 3.99329269e+00, 2.87923924e+00,
       2.04108096e+00, 1.42591446e+00, 9.83953860e-01, 6.72089622e-01,
       4.55268643e-01, 3.06333785e-01, 2.05017382e-01, 1.36625188e-01,
       9.07399281e-02, 6.01034454e-02])